In [1]:
import zipfile
import os

folder_path = 'C:\Deepak\Warm Winter 2020 ecosystem eddy covariance flux product for 73 stations in FLUXNET-Archive format—release 2022-1'
for file in os.listdir(folder_path):
    if file.endswith('.zip'):
        zip_path = os.path.join(folder_path, file)
        extract_path = os.path.join(folder_path, file[:-4])  # remove .zip
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)

In [1]:
pwd

'c:\\Deepak\\stations'

In [1]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 966C-A36B

 Directory of c:\Deepak\stations\MM\BRA

07/19/2025  11:58 AM    <DIR>          .
07/19/2025  11:29 AM    <DIR>          ..
07/19/2025  11:18 AM           784,673 FLX_BE-Bra_FLUXNET2015_FULLSET_MM_1996-2020_beta-3.csv
07/14/2025  11:18 AM           164,432 ICOSETC_BE-Bra_FLUXNET_MM_L2.csv
07/19/2025  11:49 AM             1,598 zip_extract.ipynb
               3 File(s)        950,703 bytes
               2 Dir(s)  483,477,213,184 bytes free


In [2]:
import pandas as pd

In [4]:
ds = pd.read_csv('FLX_BE-Bra_FLUXNET2015_FULLSET_MM_1996-2020_beta-3.csv')

In [5]:
type(ds)

pandas.core.frame.DataFrame

In [7]:
ds.columns.shape

(344,)

In [9]:
ds2 = pd.read_csv('ICOSETC_BE-Bra_FLUXNET_MM_L2.csv')

In [10]:
ds2.columns.shape

(340,)

In [11]:
common = set(ds.columns) & set(ds2.columns)
unique_to_ds = set(ds.columns) - set(ds2.columns)
unique_to_ds2 = set(ds2.columns) - set(ds.columns)

print("Common columns:", sorted(common))
print("\nColumns only in FLUXNET2015:", sorted(unique_to_ds))
print("\nColumns only in ICOSETC:", sorted(unique_to_ds2))

Common columns: ['CO2_F_MDS', 'CO2_F_MDS_QC', 'DAY_D', 'DAY_RANDUNC_N', 'EBC_CF_METHOD', 'EBC_CF_N', 'GPP_DT_CUT_05', 'GPP_DT_CUT_16', 'GPP_DT_CUT_25', 'GPP_DT_CUT_50', 'GPP_DT_CUT_75', 'GPP_DT_CUT_84', 'GPP_DT_CUT_95', 'GPP_DT_CUT_MEAN', 'GPP_DT_CUT_REF', 'GPP_DT_CUT_SE', 'GPP_DT_CUT_USTAR50', 'GPP_DT_VUT_05', 'GPP_DT_VUT_16', 'GPP_DT_VUT_25', 'GPP_DT_VUT_50', 'GPP_DT_VUT_75', 'GPP_DT_VUT_84', 'GPP_DT_VUT_95', 'GPP_DT_VUT_MEAN', 'GPP_DT_VUT_REF', 'GPP_DT_VUT_SE', 'GPP_DT_VUT_USTAR50', 'GPP_NT_CUT_05', 'GPP_NT_CUT_16', 'GPP_NT_CUT_25', 'GPP_NT_CUT_50', 'GPP_NT_CUT_75', 'GPP_NT_CUT_84', 'GPP_NT_CUT_95', 'GPP_NT_CUT_MEAN', 'GPP_NT_CUT_REF', 'GPP_NT_CUT_SE', 'GPP_NT_CUT_USTAR50', 'GPP_NT_VUT_05', 'GPP_NT_VUT_16', 'GPP_NT_VUT_25', 'GPP_NT_VUT_50', 'GPP_NT_VUT_75', 'GPP_NT_VUT_84', 'GPP_NT_VUT_95', 'GPP_NT_VUT_MEAN', 'GPP_NT_VUT_REF', 'GPP_NT_VUT_SE', 'GPP_NT_VUT_USTAR50', 'G_F_MDS', 'G_F_MDS_QC', 'H_CORR', 'H_F_MDS', 'H_F_MDS_QC', 'H_RANDUNC', 'LE_CORR', 'LE_F_MDS', 'LE_F_MDS_QC', 'LE_RAND

In [17]:
import pandas as pd

def process_fluxnet_data(ds1_path, ds2_path, output_path):
    # Define core variables (including TIMESTAMP first)
    core_vars = [
        'TIMESTAMP',
        # Base variables
        'LE_F_MDS', 'H_F_MDS', 'G_F_MDS', 'NETRAD', 'TA_F',
        'TS_F_MDS_1', 'VPD_F', 'P_F', 'SWC_F_MDS_1', 'GPP_NT_VUT_MEAN',
        # Corrected versions
        'LE_CORR', 'H_CORR',
        # Quality control flags
        'LE_F_MDS_QC', 'H_F_MDS_QC', 'G_F_MDS_QC', 'TA_F_QC',
        'TS_F_MDS_1_QC', 'VPD_F_QC', 'P_F_QC', 'SWC_F_MDS_1_QC',

        # Uncertainty measures
    ]

    # Load datasets with only needed columns
    ds1 = pd.read_csv(ds1_path, usecols=lambda x: x in core_vars)
    ds2 = pd.read_csv(ds2_path, usecols=lambda x: x in core_vars)

    # Convert TIMESTAMP to datetime for comparison
    ds1['TIMESTAMP'] = pd.to_datetime(ds1['TIMESTAMP'], format='%Y%m')
    ds2['TIMESTAMP'] = pd.to_datetime(ds2['TIMESTAMP'], format='%Y%m')

    # Find overlapping dates
    overlap_dates = set(ds1['TIMESTAMP']).intersection(set(ds2['TIMESTAMP']))

    # Process overlapping dates - prioritize non -9999 values
    for date in overlap_dates:
        for var in core_vars[1:]:  # Skip TIMESTAMP
            if var in ds1.columns and var in ds2.columns:
                # Get values from both datasets
                val1 = ds1.loc[ds1['TIMESTAMP'] == date, var].values[0]
                val2 = ds2.loc[ds2['TIMESTAMP'] == date, var].values[0]

                # Prioritize non -9999 values
                if val2 != -9999:
                    ds1.loc[ds1['TIMESTAMP'] == date, var] = val2
                elif val1 == -9999 and val2 == -9999:
                    # Both are -9999, keep ds1 value
                    pass

    # Remove overlapping dates from ds2 (since we've merged them into ds1)
    ds2 = ds2[~ds2['TIMESTAMP'].isin(overlap_dates)]

    # Concatenate chronologically
    final_df = pd.concat([ds1, ds2]).sort_values('TIMESTAMP')

    # Convert TIMESTAMP back to original format
    final_df['TIMESTAMP'] = final_df['TIMESTAMP'].dt.strftime('%Y%m')

    # Save to CSV
    final_df.to_csv(output_path, index=False)
    print(f"Successfully processed data. Output saved to {output_path}")
    print(f"Time coverage: {final_df['TIMESTAMP'].min()} to {final_df['TIMESTAMP'].max()}")

# Usage
process_fluxnet_data(
    'FLX_BE-Bra_FLUXNET2015_FULLSET_MM_1996-2020_beta-3.csv',
    'ICOSETC_BE-Bra_FLUXNET_MM_L2.csv',
    'BE-Bra_Fluxnet_Combined_1996-2024.csv'
)

Successfully processed data. Output saved to BE-Bra_Fluxnet_Combined_1996-2024.csv
Time coverage: 199601 to 202412


In [20]:
ds1 = pd.read_csv('BE-Bra_Fluxnet_Combined_1996-2024.csv')

In [21]:
ds1.columns

Index(['TIMESTAMP', 'TA_F', 'TA_F_QC', 'VPD_F', 'VPD_F_QC', 'P_F', 'P_F_QC',
       'NETRAD', 'TS_F_MDS_1', 'TS_F_MDS_1_QC', 'SWC_F_MDS_1',
       'SWC_F_MDS_1_QC', 'G_F_MDS', 'G_F_MDS_QC', 'LE_F_MDS', 'LE_F_MDS_QC',
       'LE_CORR', 'H_F_MDS', 'H_F_MDS_QC', 'H_CORR', 'GPP_NT_VUT_MEAN'],
      dtype='object')

In [22]:
import pandas as pd

def analyze_combined_fluxnet(csv_path):
    # Load the combined dataset
    df = pd.read_csv(csv_path)
    
    # Define QC flag meanings (standard FLUXNET coding)
    QC_FLAG_MEANINGS = {
        0: 'Measured',
        1: 'Good quality gapfill', 
        2: 'Medium quality',
        3: 'Poor quality',
        -9999: 'Missing'
    }

    print("=== Missing Value Analysis ===")
    # 1. Count missing values (-9999) for all variables
    missing_counts = {}
    for col in df.columns:
        if col != 'TIMESTAMP' and not col.endswith('_QC'):
            missing_counts[col] = (df[col] == -9999).sum()
    
    print("Missing values (-9999) count per variable:")
    for var, count in missing_counts.items():
        print(f"{var}: {count} missing ({count/len(df)*100:.1f}%)")

    print("\n=== Quality Flag Analysis ===")
    # 2. Quality flag distribution for ALL QC variables
    qc_columns = [col for col in df.columns if col.endswith('_QC')]
    
    for qc_col in qc_columns:
        print(f"\nQuality Flag Distribution for {qc_col}:")
        
        # Get distribution
        flag_dist = df[qc_col].value_counts().sort_index()
        
        # Print with meaningful labels
        for code, count in flag_dist.items():
            label = QC_FLAG_MEANINGS.get(code, f'Unknown code {code}')
            print(f"{label}: {count} ({count/len(df)*100:.1f}%)")
        
        # Calculate percentage of measured data (code 0)
        if 0 in flag_dist:
            measured_pct = flag_dist[0]/len(df)*100
            print(f"--> Measured data percentage: {measured_pct:.1f}%")

    print("\n=== Basic Statistics ===")
    # 3. General statistics for all variables (excluding QC flags and timestamp)
    numeric_cols = [col for col in df.columns if not col.endswith('_QC') and col != 'TIMESTAMP']
    print(df[numeric_cols].describe())

# Example usage
analyze_combined_fluxnet('BE-Bra_Fluxnet_Combined_1996-2024.csv')

=== Missing Value Analysis ===
Missing values (-9999) count per variable:
TA_F: 0 missing (0.0%)
VPD_F: 0 missing (0.0%)
P_F: 0 missing (0.0%)
NETRAD: 117 missing (33.6%)
TS_F_MDS_1: 12 missing (3.4%)
SWC_F_MDS_1: 228 missing (65.5%)
G_F_MDS: 42 missing (12.1%)
LE_F_MDS: 0 missing (0.0%)
LE_CORR: 32 missing (9.2%)
H_F_MDS: 0 missing (0.0%)
H_CORR: 32 missing (9.2%)
GPP_NT_VUT_MEAN: 36 missing (10.3%)

=== Quality Flag Analysis ===

Quality Flag Distribution for TA_F_QC:
Measured: 28 (8.0%)
Unknown code 0.000672043: 2 (0.6%)
Unknown code 0.00416667: 1 (0.3%)
Unknown code 0.00604839: 1 (0.3%)
Unknown code 0.202285: 1 (0.3%)
Unknown code 0.258065: 1 (0.3%)
Unknown code 0.711806: 1 (0.3%)
Unknown code 0.72043: 1 (0.3%)
Unknown code 0.735215: 1 (0.3%)
Unknown code 0.736806: 1 (0.3%)
Unknown code 0.754167: 1 (0.3%)
Unknown code 0.773522: 1 (0.3%)
Unknown code 0.784226: 1 (0.3%)
Unknown code 0.815188: 1 (0.3%)
Unknown code 0.834677: 1 (0.3%)
Unknown code 0.838889: 1 (0.3%)
Unknown code 0.8833

In [23]:
import pandas as pd
import numpy as np

def analyze_combined_fluxnet(csv_path):
    # Load the combined dataset
    df = pd.read_csv(csv_path)
    
    # Define QC flag meanings (standard FLUXNET coding)
    QC_FLAG_MEANINGS = {
        0: 'Measured (original)',
        1: 'Good quality gapfill', 
        2: 'Medium quality gapfill',
        3: 'Poor quality gapfill',
        -9999: 'Missing'
    }

    print("=== Missing Value Analysis ===")
    # 1. Count missing values (-9999) for all variables
    missing_counts = {}
    for col in df.columns:
        if col != 'TIMESTAMP' and not col.endswith('_QC'):
            missing_counts[col] = (df[col] == -9999).sum()
    
    print("Missing values (-9999) count per variable:")
    for var, count in missing_counts.items():
        print(f"{var}: {count} missing ({count/len(df)*100:.1f}%)")

    print("\n=== Quality Flag Analysis ===")
    # 2. Quality flag distribution for ALL QC variables (with floating-point handling)
    qc_columns = [col for col in df.columns if col.endswith('_QC')]
    
    for qc_col in qc_columns:
        print(f"\nQuality Flag Distribution for {qc_col}:")
        
        # Round QC values to nearest integer and replace -9999
        rounded_qc = np.where(df[qc_col] == -9999, -9999, 
                            np.round(df[qc_col]).astype(int))
        
        # Get distribution of rounded values
        flag_dist = pd.Series(rounded_qc).value_counts().sort_index()
        
        # Print with meaningful labels
        for code, count in flag_dist.items():
            label = QC_FLAG_MEANINGS.get(code, f'Unknown code {code}')
            print(f"{label}: {count} ({count/len(df)*100:.1f}%)")
        
        # Calculate percentage of measured data (code 0)
        if 0 in flag_dist:
            measured_pct = flag_dist[0]/len(df)*100
            print(f"--> Measured data percentage: {measured_pct:.1f}%")

    print("\n=== Basic Statistics ===")
    # 3. General statistics for all variables (excluding QC flags and timestamp)
    numeric_cols = [col for col in df.columns if not col.endswith('_QC') and col != 'TIMESTAMP']
    print(df[numeric_cols].describe())

# Example usage
analyze_combined_fluxnet('BE-Bra_Fluxnet_Combined_1996-2024.csv')

=== Missing Value Analysis ===
Missing values (-9999) count per variable:
TA_F: 0 missing (0.0%)
VPD_F: 0 missing (0.0%)
P_F: 0 missing (0.0%)
NETRAD: 117 missing (33.6%)
TS_F_MDS_1: 12 missing (3.4%)
SWC_F_MDS_1: 228 missing (65.5%)
G_F_MDS: 42 missing (12.1%)
LE_F_MDS: 0 missing (0.0%)
LE_CORR: 32 missing (9.2%)
H_F_MDS: 0 missing (0.0%)
H_CORR: 32 missing (9.2%)
GPP_NT_VUT_MEAN: 36 missing (10.3%)

=== Quality Flag Analysis ===

Quality Flag Distribution for TA_F_QC:
Measured (original): 34 (9.8%)
Good quality gapfill: 314 (90.2%)
--> Measured data percentage: 9.8%

Quality Flag Distribution for VPD_F_QC:
Measured (original): 46 (13.2%)
Good quality gapfill: 302 (86.8%)
--> Measured data percentage: 13.2%

Quality Flag Distribution for P_F_QC:
Measured (original): 33 (9.5%)
Good quality gapfill: 315 (90.5%)
--> Measured data percentage: 9.5%

Quality Flag Distribution for TS_F_MDS_1_QC:
Missing: 12 (3.4%)
Measured (original): 23 (6.6%)
Good quality gapfill: 313 (89.9%)
--> Measured 

In [25]:
import pandas as pd
import numpy as np
import sys

def save_fluxnet_analysis(csv_path, output_txt_path):
    # Configure pandas to show all output
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    pd.set_option('display.max_colwidth', None)
    
    # Redirect stdout to file
    original_stdout = sys.stdout
    with open(output_txt_path, 'w') as f:
        sys.stdout = f
        
        # Load the combined dataset
        df = pd.read_csv(csv_path)
        
        # Define QC flag meanings (standard FLUXNET coding)
        QC_FLAG_MEANINGS = {
            0: 'Measured (original)',
            1: 'Good quality gapfill', 
            2: 'Medium quality gapfill',
            3: 'Poor quality gapfill',
            -9999: 'Missing'
        }

        print("=== FLUXNET Data Quality Analysis Report ===")
        print(f"Dataset: {csv_path}")
        print(f"Total records: {len(df)}\n")
        
        print("=== 1. Missing Value Analysis ===")
        missing_counts = {}
        for col in df.columns:
            if col != 'TIMESTAMP' and not col.endswith('_QC'):
                missing_counts[col] = (df[col] == -9999).sum()
        
        print("Missing values (-9999) count per variable:")
        for var, count in missing_counts.items():
            print(f"{var}: {count} missing ({count/len(df)*100:.1f}%)")

        print("\n=== 2. Quality Flag Analysis ===")
        qc_columns = [col for col in df.columns if col.endswith('_QC')]
        
        for qc_col in qc_columns:
            print(f"\n-- {qc_col} --")
            
            # Round QC values to nearest integer
            rounded_qc = np.where(df[qc_col] == -9999, -9999, 
                                np.round(df[qc_col]).astype(int))
            
            flag_dist = pd.Series(rounded_qc).value_counts().sort_index()
            
            for code, count in flag_dist.items():
                label = QC_FLAG_MEANINGS.get(code, f'Unknown code {code}')
                print(f"{label}: {count} ({count/len(df)*100:.1f}%)")
            
            if 0 in flag_dist:
                measured_pct = flag_dist[0]/len(df)*100
                print(f"--> Measured data percentage: {measured_pct:.1f}%")

        print("\n=== 3. Basic Statistics ===")
        numeric_cols = [col for col in df.columns if not col.endswith('_QC') and col != 'TIMESTAMP']
        print(df[numeric_cols].describe())
        
        # Reset stdout
        sys.stdout = original_stdout
    
    print(f"Complete analysis saved to: {output_txt_path}")

# Usage
save_fluxnet_analysis(
    'BE-Bra_Fluxnet_Combined_1996-2024.csv',
    'BE-Bra_Fluxnet_Analysis_Report.txt'
)

Complete analysis saved to: BE-Bra_Fluxnet_Analysis_Report.txt


In [26]:
import pandas as pd
import numpy as np
import sys

def save_fluxnet_analysis(csv_path, output_txt_path):
    # Configure pandas to show all output
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    pd.set_option('display.max_colwidth', None)
    
    # Redirect stdout to file
    original_stdout = sys.stdout
    with open(output_txt_path, 'w') as f:
        sys.stdout = f
        
        # Load the combined dataset
        df = pd.read_csv(csv_path)
        
        # Define QC flag meanings
        QC_FLAG_MEANINGS = {
            0: 'Measured (original)',
            1: 'Good quality gapfill', 
            2: 'Medium quality gapfill',
            3: 'Poor quality gapfill',
            -9999: 'Missing'
        }

        print("=== FLUXNET Data Quality Analysis Report ===")
        print(f"Dataset: {csv_path}")
        print(f"Total records: {len(df)}\n")
        
        print("=== 1. Missing Value Analysis ===")
        missing_counts = {}
        zero_counts = {}
        for col in df.columns:
            if col != 'TIMESTAMP' and not col.endswith('_QC'):
                missing_counts[col] = (df[col] == -9999).sum()
                zero_counts[col] = ((df[col] == 0) & (df[col] != -9999)).sum()
        
        print("\nMissing values (-9999) count per variable:")
        for var, count in missing_counts.items():
            print(f"{var}: {count} missing ({count/len(df)*100:.1f}%)")
        
        print("\nZero values (0.0) count per variable (excluding missing):")
        for var, count in zero_counts.items():
            print(f"{var}: {count} zeros ({count/len(df)*100:.1f}%)")

        print("\n=== 2. Quality Flag Analysis ===")
        qc_columns = [col for col in df.columns if col.endswith('_QC')]
        
        for qc_col in qc_columns:
            print(f"\n-- {qc_col} --")
            
            # Round QC values to nearest integer
            rounded_qc = np.where(df[qc_col] == -9999, -9999, 
                                np.round(df[qc_col]).astype(int))
            
            flag_dist = pd.Series(rounded_qc).value_counts().sort_index()
            
            for code, count in flag_dist.items():
                label = QC_FLAG_MEANINGS.get(code, f'Unknown code {code}')
                print(f"{label}: {count} ({count/len(df)*100:.1f}%)")
            
            if 0 in flag_dist:
                measured_pct = flag_dist[0]/len(df)*100
                print(f"--> Measured data percentage: {measured_pct:.1f}%")

        print("\n=== 3. Basic Statistics ===")
        numeric_cols = [col for col in df.columns if not col.endswith('_QC') and col != 'TIMESTAMP']
        print(df[numeric_cols].describe())
        
        # Reset stdout
        sys.stdout = original_stdout
    
    print(f"Complete analysis saved to: {output_txt_path}")

# Usage
save_fluxnet_analysis(
    'BE-Bra_Fluxnet_Combined_1996-2024.csv',
    'BE-Bra_Fluxnet_Analysis_Report.txt'
)

Complete analysis saved to: BE-Bra_Fluxnet_Analysis_Report.txt
